# Explainer for Iris model with Poetry-defined Environment

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * poetry
 * rclone
 * curl

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Install Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

We will assume that ambassador (or Istio) ingress is port-forwarded to `localhost:8003`

## Setup MinIO

Use the provided [notebook](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html) to install Minio in your cluster.
Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/minio_setup.html).

We will assume that MinIO service is port-forwarded to `localhost:8090`

In [1]:
%%writefile rclone.conf
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://localhost:8090

Overwriting rclone.conf


In [2]:
%%writefile secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: seldon-rclone-secret
type: Opaque
stringData:
  RCLONE_CONFIG_S3_TYPE: s3
  RCLONE_CONFIG_S3_PROVIDER: minio
  RCLONE_CONFIG_S3_ENV_AUTH: "false"
  RCLONE_CONFIG_S3_ACCESS_KEY_ID: minioadmin
  RCLONE_CONFIG_S3_SECRET_ACCESS_KEY: minioadmin
  RCLONE_CONFIG_S3_ENDPOINT: http://minio.minio-system.svc.cluster.local:9000

Overwriting secret.yaml


In [3]:
!kubectl apply -f secret.yaml

secret/seldon-rclone-secret configured


## Poetry

We will use `poetry.lock` to fully define the explainer environment. Install poetry following official [documentation](https://python-poetry.org/docs/#installation). Usually this goes down to
```bash
curl -sSL https://raw.githubusercontent.com/python-poetry/poetry/master/get-poetry.py | python3 -
```

# Deploy Iris Model

In [80]:
%%writefile iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: iris
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/v1.11.0-dev/sklearn/iris

Overwriting iris.yaml


In [81]:
!kubectl apply -f iris.yaml

seldondeployment.machinelearning.seldon.io/iris configured


In [82]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=iris -o jsonpath='{.items[0].metadata.name}')

deployment "iris-default-0-classifier" successfully rolled out


In [83]:
%%bash
curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:8003/seldon/seldon/iris/api/v1.0/predictions  | jq .

{
  "data": {
    "names": [
      "t:0",
      "t:1",
      "t:2"
    ],
    "ndarray": [
      [
        0.9548873249364059,
        0.04505474761562512,
        5.7927447968953825e-05
      ]
    ]
  },
  "meta": {
    "requestPath": {
      "classifier": "seldonio/sklearnserver:1.12.0-dev"
    }
  }
}


In [90]:
%%bash
kubectl exec  iris-default-0-classifier-7c5b78594-bw52s --  \
   curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://localhost:9000/api/v1.0/predictions

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.9548873249364059,0.04505474761562512,5.7927447968953825e-05]]},"meta":{"requestPath":{"classifier":"seldonio/sklearnserver:1.12.0-dev"}}}


Defaulting container name to classifier.
Use 'kubectl describe pod/iris-default-0-classifier-7c5b78594-bw52s -n seldon' to see all of the containers in this pod.


In [89]:
%%bash
kubectl run --quiet=true -it --rm curl --image=radial/busyboxplus:curl --restart=Never --  \
   curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://iris-default.seldon.svc.cluster.local:8000/api/v1.0/predictions

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.9548873249364059,0.04505474761562512,5.7927447968953825e-05]]},"meta":{"requestPath":{"classifier":"seldonio/sklearnserver:1.12.0-dev"}}}


In [86]:
%%bash
kubectl run --quiet=true -it --rm curl --image=radial/busyboxplus:curl --restart=Never --  \
   curl -s -X POST -H 'Content-Type: application/json' \
    -d '{"data":{"ndarray":[[5.964, 4.006, 2.081, 1.031]]}}' \
    http://istio-ingressgateway.istio-system.svc.cluster.local/seldon/seldon/iris/api/v1.0/predictions

{"data":{"names":["t:0","t:1","t:2"],"ndarray":[[0.9548873249364059,0.04505474761562512,5.7927447968953825e-05]]},"meta":{"requestPath":{"classifier":"seldonio/sklearnserver:1.12.0-dev"}}}


# Train Explainer

In [126]:
import numpy as np
from sklearn.datasets import load_iris
from alibi.explainers import AnchorTabular

import requests

In [127]:
dataset = load_iris()
feature_names = dataset.feature_names
iris_data = dataset.data

In [128]:
model_url = "http://localhost:8003/seldon/seldon/iris/api/v1.0/predictions"


def predict_fn(X):
    data = {"data": {"ndarray": X.tolist()}}
    r = requests.post(model_url, json={"data": {"ndarray": [[1, 2, 3, 4]]}})
    return np.array(r.json()["data"]["ndarray"])

In [129]:
predict_fn(np.array([[5.964, 4.006, 2.081, 1.031]]))

array([[6.98519453e-04, 3.66803904e-03, 9.95633442e-01]])

In [130]:
explainer = AnchorTabular(predict_fn, feature_names)

In [131]:
explainer.fit(iris_data, disc_perc=(25, 50, 75))

AnchorTabular(meta={
  'name': 'AnchorTabular',
  'type': ['blackbox'],
  'explanations': ['local'],
  'params': {'disc_perc': (25, 50, 75), 'seed': None}}
)

# Save and deploy Explainer

In [132]:
explainer.save("./explainer/")

In [97]:
!tree

.
├── explainer
│   ├── explainer.dill
│   └── meta.dill
├── iris-with-explainer.yaml
├── iris.yaml
├── rclone.conf
├── README.ipynb
└── secret.yaml

1 directory, 7 files


In [98]:
!rclone --config="rclone.conf" copy explainer/ s3:explainers/iris/

In [133]:
%%writefile iris-with-explainer.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: iris
spec:
  predictors:
  - name: default
    replicas: 1
    graph:
      name: classifier
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/v1.11.0-dev/sklearn/iris
    explainer:
      type: AnchorTabular
      modelUri: s3:explainers/iris/
      envSecretRefName: seldon-rclone-secret
      replicas: 1            

Overwriting iris-with-explainer.yaml


In [134]:
!kubectl apply -f iris-with-explainer.yaml

seldondeployment.machinelearning.seldon.io/iris unchanged


# Test Deployed explainer

In [135]:
model_url = "http://localhost:8003/seldon/seldon/iris-explainer/default/api/v1.0/explain"


def explain_fn(X):
    data = {"data": {"ndarray": X.tolist()}}
    r = requests.post(model_url, json={"data": {"ndarray": [[1, 2, 3, 4]]}})
    return r.json()

In [136]:
explanation = explain_fn(np.array([[5.964, 4.006, 2.081, 1.031]]))

In [137]:
print('Anchor: %s' % (' AND '.join(explanation["data"]["anchor"])))
print('Precision: %.2f' % explanation["data"]["precision"])
print('Coverage: %.2f' % explanation["data"]["coverage"])

Anchor: petal width (cm) > 1.80 AND sepal width (cm) <= 2.80
Precision: 0.98
Coverage: 0.08
